# Imports

In [1]:
import numpy as np

# Helper Functions

In [2]:
def generateBitstrings(n):
    bitstrings = []
    for i in range(2**n):
        bitstrings.append(f'{i:0{n}b}')
    return bitstrings

def addBitstrings(x, y):
    z = []
    for i, bit in enumerate(x):
        z.append(str((int(bit) + int(y[i])) % 2))
    return ''.join(z)

def generateF(bitstrings, s):
    f = {}
    for x in bitstrings:
        if x not in f:
            y = x if x[-1] == '0' else addBitstrings(x, s)
            f[y] = y
            f[addBitstrings(y, s)] = y
    return f

def hammingDistance(x, y):
    dist = 0
    for i, bit in enumerate(x):
        if bit != y[i]:
            dist += 1
    return dist

def bitstringToVector(x):
    v = np.zeros(2**len(x))
    v[int(x, 2)] = 1
    return v

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)

# Create Hamiltonian

In [3]:
def hamiltonian(n, s=None, verbose=False):
    outerBitstrings = generateBitstrings(n)
    innerBitstrings = generateBitstrings(n-1)
    if s == None:
        s = outerBitstrings[-1]
    f = generateF(outerBitstrings, s)
    outerSum = np.zeros((2**(2*n-1), 2**(2*n-1)))
    for x in outerBitstrings:
        v = bitstringToVector(x)
        innerSum = np.zeros((2**(n-1), 2**(n-1)))
        for y in innerBitstrings:
            w = bitstringToVector(y)
            innerSum += hammingDistance(y, f[x]) * np.outer(w, w)
        outerSum += np.kron(np.outer(v, v), innerSum)
    if verbose:
        print('Outer bitstrings:', outerBitstrings)
        print('Inner bitstrings:', innerBitstrings)
        print('Function f(x):   ', f)
    return outerSum

qubo = hamiltonian(2, verbose=True)
qubo

Outer bitstrings: ['00', '01', '10', '11']
Inner bitstrings: ['0', '1']
Function f(x):    {'00': '00', '11': '00', '10': '10', '01': '10'}


array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

# Solve QUBO

In [4]:
def solveQubo(q, n):
    mins = set()
    minExpectation = np.inf
    for i in range(2**((2**n)*(2**(n-1)))):
        x = f'{i:0{(2**n)*(2**(n-1))}b}'
        v = np.array(list(map(int, list(x))))
        expectation = v.T @ q @ v
        # if expectation <= 0:
        #     print(x, expectation)
        if expectation < minExpectation:
            minExpectation = expectation
            mins = set()
            mins.add(x)
        if expectation == minExpectation:
            mins.add(x)
    return mins, minExpectation

# def interpretResults(results, n):
#     candidates = set()
#     decodedResults = set()
#     for result in results:
#         indices = list(find_all(result, '1'))
#         for index in indices:
#             if index not in candidates:
#                 print(f'{index:0{2*n-1}b}')
#                 candidates.add(index)
#             decodedResults.add(f'{index:0{2*n-1}b}'[n:])
#     return decodedResults

results, expectation = solveQubo(qubo, 2)
results

{'00000000',
 '00000010',
 '00000100',
 '00000110',
 '00010000',
 '00010010',
 '00010100',
 '00010110',
 '10000000',
 '10000010',
 '10000100',
 '10000110',
 '10010000',
 '10010010',
 '10010100',
 '10010110'}